In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display_html
import seaborn as sns
from pandas.plotting import scatter_matrix

sns.set()

%matplotlib inline

import tjse

## Experimento 1

In [ ]:
experimento1 = tjse.carregar_dados_experimento1()

experimento1.head()

,Chave,TUD,Stemmer,Coleção
0,201100200560443632531,576,NoStem,Segundo Grau - Acórdãos
1,201100200560443632531,517,Porter,Segundo Grau - Acórdãos
2,201100200560443632531,480,RSLP,Segundo Grau - Acórdãos
3,201100200560443632531,540,RSLP-S,Segundo Grau - Acórdãos
4,201100200560443632531,535,UniNE,Segundo Grau - Acórdãos


In [ ]:
plt.figure(figsize=(12, 6))
graph1 = sns.pointplot(x="Stemmer", y="TUD", hue="Coleção",
                 data=experimento1, n_boot=1000)
graph1.set_ylabel('Média(TUD)')
graph1.set_xlabel('Algoritmo')
graph1.set_title('Termos únicos por Documento')
plt.show()

In [ ]:
def percentual_de_reducao(dados):
    resultado = pd.pivot_table(dados, index=['Coleção', 'Chave'], columns='Stemmer', values='TUD') 
    
    resultado = resultado.assign(Porter = 100 - (resultado['Porter'] * 100) / resultado['NoStem'],
                                RSLPS = 100 - (resultado['RSLP-S'] * 100) / resultado['NoStem'],
                                RSLP = 100 - (resultado['RSLP'] * 100) / resultado['NoStem'],
                                UniNE = 100 - (resultado['UniNE'] * 100) / resultado['NoStem'],
                                NoStemTUD = resultado['NoStem'],
                                NoStem = 0.0)
    resultado.drop(['RSLP-S'], axis=1, inplace=True)
    resultado.rename(columns={'RSLPS': 'RSLP-S'}, inplace=True)
    
    return resultado

df_pr = percentual_de_reducao(experimento1)
df_graph = pd.melt(df_pr.reset_index(),
                   id_vars = ['Chave', 'Coleção'], 
                   value_vars=['NoStem','Porter','RSLP','RSLP-S','UniNE'], value_name='PR')
plt.figure(figsize=(12, 6))
graph2 = sns.pointplot(x="Stemmer", y="PR", hue="Coleção",
                 data=df_graph, n_boot=1000)
graph2.set_ylabel('Média(PR)')
graph2.set_xlabel('Algoritmo')
graph2.set_title('Percentual de Redução dos Termos')
plt.show()

In [ ]:
df_corr_graph = df_pr.drop(['NoStem'], axis=1).rename(columns={'NoStemTUD': 'NoStem'})
df_corr_graph = df_corr_graph[['NoStem', 'Porter', 'RSLP', 'RSLP-S', 'UniNE']]
axes = scatter_matrix(df_corr_graph, alpha=0.2, figsize=(12, 12), diagonal='kde')
corr = df_corr_graph.corr().as_matrix()
for i, j in zip(*plt.np.triu_indices_from(axes, k=1)):
    axes[i, j].annotate("%.2f" %corr[i,j], (0.8, 0.8), xycoords='axes fraction', ha='center', va='center')
plt.show()

## Experimento 2